In [3]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import os, sys
sys.path.append('..')

from scripts.modeling.mlflow_config import *
from scripts.modeling.model_factory import *
from scripts.modeling.model_trainer import *
from scripts.modeling.data_preparation import *
from scripts.data_utils.loaders import load_data

DATAPATH = os.path.join('..', 'resources', 'data')

In [7]:
# Load datasets
fraud_data_path = os.path.join(DATAPATH, 'processed', "fraud_processed.csv")
credit_card_path = os.path.join(DATAPATH, "creditcard.csv")

fraud_df = load_data(fraud_data_path)
credit_df = load_data(credit_card_path)
fraud_df.info(), credit_df.info()

2025-02-19 09:41:44 - [✅ INFO] - Loading data from ..\resources\data\processed\fraud_processed.csv
2025-02-19 09:41:46 - [✅ INFO] - Successfully loaded data from ..\resources\data\processed\fraud_processed.csv
2025-02-19 09:41:46 - [✅ INFO] - Loading data from ..\resources\data\creditcard.csv
2025-02-19 09:41:53 - [✅ INFO] - Successfully loaded data from ..\resources\data\creditcard.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151112 entries, 0 to 151111
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   user_id                   151112 non-null  int64  
 1   signup_time               151112 non-null  object 
 2   purchase_time             151112 non-null  object 
 3   purchase_value            151112 non-null  float64
 4   device_id                 151112 non-null  object 
 5   source                    151112 non-null  int64  
 6   browser                   151112 non-null  int

(None, None)

In [8]:
fraud_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151112 entries, 0 to 151111
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   user_id                   151112 non-null  int64  
 1   signup_time               151112 non-null  object 
 2   purchase_time             151112 non-null  object 
 3   purchase_value            151112 non-null  float64
 4   device_id                 151112 non-null  object 
 5   source                    151112 non-null  int64  
 6   browser                   151112 non-null  int64  
 7   sex                       151112 non-null  int64  
 8   age                       151112 non-null  int64  
 9   ip_address                151112 non-null  int64  
 10  class                     151112 non-null  int64  
 11  country                   151112 non-null  int64  
 12  hour_of_day               151112 non-null  float64
 13  day_of_week               151112 non-null  f

In [9]:
fraud_target = 'class'
drop_columns = ['user_id', 'signup_time', 'purchase_time', 'device_id', 'ip_address']
X_fraud, y_fraud = preprocess_data(fraud_df, fraud_target, drop_columns)
X_fraud, y_fraud

(        purchase_value  source  browser  sex  age  country  hour_of_day  \
 0             0.310345       2        0    0   25      171     0.434783   
 1             0.220690       1        4    0   38      157     0.913043   
 2             0.262069       2        0    1   25      171     0.478261   
 3             0.365517       2        1    1   21      172     1.000000   
 4             0.179310       0        4    1   19      107     0.869565   
 ...                ...     ...      ...  ...  ...      ...          ...   
 151107        0.503448       1        4    1   59       60     0.304348   
 151108        0.075862       1        2    0   37       64     0.304348   
 151109        0.165517       1        1    0   35       84     1.000000   
 151110        0.393103       1        0    1   48      171     0.869565   
 151111        0.337931       0        0    1   25      171     0.260870   
 
         day_of_week  time_diff  transaction_velocity  \
 0          0.833333        0

In [10]:
credit_target = 'Class'
X_credit, y_credit = preprocess_data(credit_df, credit_target)
X_credit, y_credit

(            Time         V1         V2        V3        V4        V5  \
 0            0.0  -1.359807  -0.072781  2.536347  1.378155 -0.338321   
 1            0.0   1.191857   0.266151  0.166480  0.448154  0.060018   
 2            1.0  -1.358354  -1.340163  1.773209  0.379780 -0.503198   
 3            1.0  -0.966272  -0.185226  1.792993 -0.863291 -0.010309   
 4            2.0  -1.158233   0.877737  1.548718  0.403034 -0.407193   
 ...          ...        ...        ...       ...       ...       ...   
 284802  172786.0 -11.881118  10.071785 -9.834783 -2.066656 -5.364473   
 284803  172787.0  -0.732789  -0.055080  2.035030 -0.738589  0.868229   
 284804  172788.0   1.919565  -0.301254 -3.249640 -0.557828  2.630515   
 284805  172788.0  -0.240440   0.530483  0.702510  0.689799 -0.377961   
 284806  172792.0  -0.533413  -0.189733  0.703337 -0.506271 -0.012546   
 
               V6        V7        V8        V9  ...       V20       V21  \
 0       0.462388  0.239599  0.098698  0.36378

In [11]:
X_fraud_train, X_fraud_test, y_fraud_train, y_fraud_test = stratified_split(X_fraud, y_fraud)


In [12]:
X_credit_train, X_credit_test, y_credit_train, y_credit_test = stratified_split(X_credit, y_credit)


In [19]:
configure_mlflow()

2025/02/19 10:01:10 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/02/19 10:01:10 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

In [22]:
fraud_trainer = ModelTrainer(X_fraud_train, X_fraud_test, y_fraud_train, y_fraud_test, 'fraud')
fraud_trainer

2025/02/19 10:04:49 INFO mlflow.tracking.fluent: Experiment with name 'Fraud_Detection_fraud' does not exist. Creating a new experiment.


In [23]:
# Traditional ML models
fraud_models = [
    ('LogisticRegression', ModelFactory.logistic_regression()),
    ('DecisionTree', DecisionTreeClassifier(class_weight='balanced')),
    ('RandomForest', ModelFactory.random_forest()),
    ('GradientBoosting', GradientBoostingClassifier(n_estimators=100))
]

for name, model in fraud_models:
    fraud_trainer.train_sklearn_model(model, name)


2025/02/19 10:04:56 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Darkles\10academy\env\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/02/19 10:05:04 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Darkles\10academy\env\Lib\site-packages\

In [24]:
# Neural Networks
fraud_nn_models = {
    'MLP': ModelFactory.mlp,
    'CNN': ModelFactory.cnn,
    'LSTM': ModelFactory.lstm
}

for name, builder in fraud_nn_models.items():
    fraud_trainer.train_keras_model(builder, name, epochs=5)

c:\Users\Darkles\10academy\env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025/02/19 10:07:35 WARNING mlflow.keras.autologging: Failed to log dataset information to MLflow. Reason: 'Series' object has no attribute 'flatten'


Epoch 1/5
378/378 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.5605 - loss: 0.7674 - val_accuracy: 0.3320 - val_loss: 0.8366
Epoch 2/5
378/378 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6758 - loss: 0.6448 - val_accuracy: 0.8154 - val_loss: 0.5211
Epoch 3/5
378/378 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6981 - loss: 0.6302 - val_accuracy: 0.6601 - val_loss: 0.6261
Epoch 4/5
378/378 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7114 - loss: 0.6118 - val_accuracy: 0.6591 - val_loss: 0.6389
Epoch 5/5
378/378 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7504 - loss: 0.5927 - val_accuracy: 0.6295 - val_loss: 0.6886
945/945 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


2025/02/19 10:08:10 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/02/19 10:08:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/19 10:08:28 WARNING mlflow.keras.autologging: Failed to log dataset information to MLflow. Reason: 'Series' object has no attribute 'flatten'


Epoch 1/5
378/378 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.5940 - loss: 0.9306 - val_accuracy: 0.9285 - val_loss: 0.3052
Epoch 2/5
378/378 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7602 - loss: 0.5693 - val_accuracy: 0.8727 - val_loss: 0.4313
Epoch 3/5
378/378 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8182 - loss: 0.5426 - val_accuracy: 0.8526 - val_loss: 0.5880
Epoch 4/5
378/378 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8483 - loss: 0.5310 - val_accuracy: 0.9061 - val_loss: 0.4075
Epoch 5/5
378/378 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8559 - loss: 0.5211 - val_accuracy: 0.9193 - val_loss: 0.3932
945/945 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


2025/02/19 10:09:04 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/02/19 10:09:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/19 10:09:21 WARNING mlflow.keras.autologging: Failed to log dataset information to MLflow. Reason: 'Series' object has no attribute 'flatten'


Epoch 1/5
378/378 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.5605 - loss: 0.6867 - val_accuracy: 0.6340 - val_loss: 0.6666
Epoch 2/5
378/378 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7536 - loss: 0.6037 - val_accuracy: 0.9273 - val_loss: 0.4391
Epoch 3/5
378/378 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7839 - loss: 0.5797 - val_accuracy: 0.8382 - val_loss: 0.5221
Epoch 4/5
378/378 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7946 - loss: 0.5722 - val_accuracy: 0.8996 - val_loss: 0.4708
Epoch 5/5
378/378 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8040 - loss: 0.5676 - val_accuracy: 0.8204 - val_loss: 0.5244
945/945 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


2025/02/19 10:09:54 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/02/19 10:10:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [ ]:
# Best model info
fraud_trainer.best_model_name, fraud_trainer.best_score

In [25]:
credit_trainer = ModelTrainer(X_credit_train, X_credit_test, y_credit_train, y_credit_test, 'credit')
credit_trainer

2025/02/19 10:10:12 INFO mlflow.tracking.fluent: Experiment with name 'Fraud_Detection_credit' does not exist. Creating a new experiment.


In [26]:
# Traditional ML models
credit_models = [
    ('LogisticRegression', ModelFactory.logistic_regression()),
    ('DecisionTree', DecisionTreeClassifier(class_weight='balanced')),
    ('RandomForest', ModelFactory.random_forest()),
    ('GradientBoosting', GradientBoostingClassifier(n_estimators=100))
]

for name, model in credit_models:
    credit_trainer.train_sklearn_model(model, name)
    

2025/02/19 10:10:14 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Darkles\10academy\env\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/02/19 10:10:21 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Darkles\10academy\env\Lib\site-packages\

In [27]:
# Neural Networks
credit_nn_models = {
    'MLP': ModelFactory.mlp,
    'CNN': ModelFactory.cnn,
    'LSTM': ModelFactory.lstm
}

for name, builder in credit_nn_models.items():
    credit_trainer.train_keras_model(builder, name, epochs=30)

c:\Users\Darkles\10academy\env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025/02/19 10:12:57 WARNING mlflow.keras.autologging: Failed to log dataset information to MLflow. Reason: 'Series' object has no attribute 'flatten'


Epoch 1/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.5493 - loss: 0.7954 - val_accuracy: 0.9165 - val_loss: 0.4916
Epoch 2/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6863 - loss: 0.6380 - val_accuracy: 0.2750 - val_loss: 1.0182
Epoch 3/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.6897 - loss: 0.6263 - val_accuracy: 0.7250 - val_loss: 0.5778
Epoch 4/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.7191 - loss: 0.6065 - val_accuracy: 0.9303 - val_loss: 0.3955
Epoch 5/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7152 - loss: 0.6179 - val_accuracy: 0.6271 - val_loss: 0.6810
Epoch 6/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7274 - loss: 0.6016 - val_accuracy: 0.4206 - val_loss: 0.8590
Epoch 7/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7574 - loss: 0.5888 - val_accuracy: 0.7796 - val_loss: 0.5615
Epoch 8/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7833 - loss: 0.5744 - val_accuracy: 0.

2025/02/19 10:13:46 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/02/19 10:14:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/19 10:14:08 WARNING mlflow.keras.autologging: Failed to log dataset information to MLflow. Reason: 'Series' object has no attribute 'flatten'


Epoch 1/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.5838 - loss: 0.8109 - val_accuracy: 0.6225 - val_loss: 0.6710
Epoch 2/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.7034 - loss: 0.6129 - val_accuracy: 0.7356 - val_loss: 0.6163
Epoch 3/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.7613 - loss: 0.5729 - val_accuracy: 0.8775 - val_loss: 0.4045
Epoch 4/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.7987 - loss: 0.5539 - val_accuracy: 0.9050 - val_loss: 0.3774
Epoch 5/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.8658 - loss: 0.5282 - val_accuracy: 0.8832 - val_loss: 0.5421
Epoch 6/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.8742 - loss: 0.5104 - val_accuracy: 0.9183 - val_loss: 0.4103
Epoch 7/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.8792 - loss: 0.5053 - val_accuracy: 0.9223 - val_loss: 0.4101
Epoch 8/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8833 - loss: 0.5061 - val_accuracy

2025/02/19 10:15:15 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/02/19 10:15:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/19 10:15:37 WARNING mlflow.keras.autologging: Failed to log dataset information to MLflow. Reason: 'Series' object has no attribute 'flatten'


Epoch 1/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.6013 - loss: 0.6839 - val_accuracy: 0.5102 - val_loss: 0.7172
Epoch 2/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7415 - loss: 0.6068 - val_accuracy: 0.8416 - val_loss: 0.5183
Epoch 3/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7821 - loss: 0.5752 - val_accuracy: 0.8550 - val_loss: 0.5059
Epoch 4/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7803 - loss: 0.5737 - val_accuracy: 0.7273 - val_loss: 0.5949
Epoch 5/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.8020 - loss: 0.5560 - val_accuracy: 0.7565 - val_loss: 0.5826
Epoch 6/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.8113 - loss: 0.5533 - val_accuracy: 0.8603 - val_loss: 0.4886
Epoch 7/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8327 - loss: 0.5470 - val_accuracy: 0.7997 - val_loss: 0.5395
Epoch 8/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8435 - loss: 0.5393 - val_accuracy: 0.

2025/02/19 10:16:58 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/02/19 10:17:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [ ]:
# Best model info
credit_trainer.best_model_name, credit_trainer.best_score